## Imports

In [17]:
import os
import numpy as np
import pandas as pd
import clustering

## Datasets

In [16]:
DATA_BASE_PATH = "./data"

FAC_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fac")
FOU_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fou")
KAR_FILE = os.path.join(DATA_BASE_PATH, "mfeat-kar")

fac = np.loadtxt(FAC_FILE, dtype=int)
fou = np.loadtxt(FOU_FILE, dtype=float)
kar = np.loadtxt(KAR_FILE, dtype=float)

## Importando melhor resultado

In [18]:
best_result = clustering.import_best_result("data/melhor_resultado_todas.pickle")
partition, _ = clustering.get_hard_patitions(best_result["membership_degree"])


## Probabilidade à priori das classes

In [19]:
Pw = np.array([len(c)/2000 for c in partition])
Pw, Pw.sum()

(array([0.1205, 0.103 , 0.0665, 0.2925, 0.0015, 0.1605, 0.    , 0.1405,
        0.0165, 0.0985]),
 1.0)

In [5]:
for i, idxs in enumerate(partition):
    pd.DataFrame(fac[idxs]).to_csv(f"data/fac{i}.csv",decimal=",", index=False, header=False)

In [9]:
pd.DataFrame(partition).to_csv(f"data/partition.csv",decimal=",", index=False, header=False)

## Probabilidade à priori dos exemplos

In [13]:
def calc_p_x_w(xk, d, means, var, cov_matrix):
    coef = np.power(2*np.pi, -d/2) 
#     print(cov_matrix)
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    (sign, logdet) = np.linalg.slogdet(inv_cov_matrix)
    sqrt_det_inv_cov = np.sqrt(sign*np.exp(logdet))
    diff = xk - means
    exp_exp = np.dot((-1/2)*np.dot(diff.T, inv_cov_matrix), diff)
    exp_func = np.exp(exp_exp)
    
    return coef * sqrt_det_inv_cov * exp_func

# def calc_p_x_w(xk, d, means, var, cov_matrix):
#     coef = math.pow(2*math.pi, d/2) 
#     inv_cov_matrix = np.linalg.inv(cov_matrix)
#     det_inv =  np.linalg.det(inv_cov_matrix)
#     diff = xk - means
#     exp_exp = np.dot(np.dot((-1/2)*(diff.T), inv_cov_matrix), diff)
#     exp_func = np.exp(exp_exp)
    
#     return exp_func/(coef * math.pow(det_inv, 1/2))

def calc_gaussian_bayesian_data(x, partition):
    n,d = x.shape
    qtd_w = len(partition)
    means = np.array([x[idxs].mean(axis=0) for idxs in partition])
    var = np.array([((x[idxs]-means[i])**2).mean(axis=0) for i, idxs in enumerate(partition)])
    cov_matrix = [np.zeros((d,d)) for _ in range(qtd_w)]

    for i in range(qtd_w):
        np.fill_diagonal(cov_matrix[i], var[i])
                
    p_x_w = np.empty((qtd_w, n))
    
    for i in range(qtd_w):
        for k in range(n):
            p_x_w[i, k] = calc_p_x_w(x[k], d, means[i], var[i], cov_matrix[i])
            
    
    return p_x_w


## Densidades por dataset

In [14]:
fac_p_x_w = calc_gaussian_bayesian_data(fac, partition)
fou_p_x_w = calc_gaussian_bayesian_data(fou, partition)
kar_p_x_w = calc_gaussian_bayesian_data(kar, partition)

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: Mean of empty slice.
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: Mean of empty slice.
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


[[0.02721726 0.         0.         ... 0.         0.         0.        ]
 [0.         0.01955221 0.         ... 0.         0.         0.        ]
 [0.         0.         0.00494736 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03042215 0.         0.        ]
 [0.         0.         0.         ... 0.         0.018136   0.        ]
 [0.         0.         0.         ... 0.         0.         0.02831604]]
[[0.02721726 0.         0.         ... 0.         0.         0.        ]
 [0.         0.01955221 0.         ... 0.         0.         0.        ]
 [0.         0.         0.00494736 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03042215 0.         0.        ]
 [0.         0.         0.         ... 0.         0.018136   0.        ]
 [0.         0.         0.         ... 0.         0.         0.02831604]]
[[0.02721726 0.         0.         ... 0.         0.         0.        ]
 [0.         0.01955221 0.         ... 

[[0.02721726 0.         0.         ... 0.         0.         0.        ]
 [0.         0.01955221 0.         ... 0.         0.         0.        ]
 [0.         0.         0.00494736 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03042215 0.         0.        ]
 [0.         0.         0.         ... 0.         0.018136   0.        ]
 [0.         0.         0.         ... 0.         0.         0.02831604]]
[[0.02721726 0.         0.         ... 0.         0.         0.        ]
 [0.         0.01955221 0.         ... 0.         0.         0.        ]
 [0.         0.         0.00494736 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03042215 0.         0.        ]
 [0.         0.         0.         ... 0.         0.018136   0.        ]
 [0.         0.         0.         ... 0.         0.         0.02831604]]
[[0.02721726 0.         0.         ... 0.         0.         0.        ]
 [0.         0.01955221 0.         ... 

[[0.02721726 0.         0.         ... 0.         0.         0.        ]
 [0.         0.01955221 0.         ... 0.         0.         0.        ]
 [0.         0.         0.00494736 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03042215 0.         0.        ]
 [0.         0.         0.         ... 0.         0.018136   0.        ]
 [0.         0.         0.         ... 0.         0.         0.02831604]]
[[0.02721726 0.         0.         ... 0.         0.         0.        ]
 [0.         0.01955221 0.         ... 0.         0.         0.        ]
 [0.         0.         0.00494736 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03042215 0.         0.        ]
 [0.         0.         0.         ... 0.         0.018136   0.        ]
 [0.         0.         0.         ... 0.         0.         0.02831604]]
[[0.02721726 0.         0.         ... 0.         0.         0.        ]
 [0.         0.01955221 0.         ... 

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


...

[[0.02856736 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0312035  0.         ... 0.         0.         0.        ]
 [0.         0.         0.02585518 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03374591 0.         0.        ]
 [0.         0.         0.         ... 0.         0.01622208 0.        ]
 [0.         0.         0.         ... 0.         0.         0.01392336]]
[[0.02856736 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0312035  0.         ... 0.         0.         0.        ]
 [0.         0.         0.02585518 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03374591 0.         0.        ]
 [0.         0.         0.         ... 0.         0.01622208 0.        ]
 [0.         0.         0.         ... 0.         0.         0.01392336]]
[[0.02856736 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0312035  0.         ... 

[[0.02856736 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0312035  0.         ... 0.         0.         0.        ]
 [0.         0.         0.02585518 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03374591 0.         0.        ]
 [0.         0.         0.         ... 0.         0.01622208 0.        ]
 [0.         0.         0.         ... 0.         0.         0.01392336]]
[[0.02856736 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0312035  0.         ... 0.         0.         0.        ]
 [0.         0.         0.02585518 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03374591 0.         0.        ]
 [0.         0.         0.         ... 0.         0.01622208 0.        ]
 [0.         0.         0.         ... 0.         0.         0.01392336]]
[[0.02856736 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0312035  0.         ... 

[[0.02856736 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0312035  0.         ... 0.         0.         0.        ]
 [0.         0.         0.02585518 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03374591 0.         0.        ]
 [0.         0.         0.         ... 0.         0.01622208 0.        ]
 [0.         0.         0.         ... 0.         0.         0.01392336]]
[[0.02856736 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0312035  0.         ... 0.         0.         0.        ]
 [0.         0.         0.02585518 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03374591 0.         0.        ]
 [0.         0.         0.         ... 0.         0.01622208 0.        ]
 [0.         0.         0.         ... 0.         0.         0.01392336]]
[[0.02856736 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0312035  0.         ... 

[[0.02856736 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0312035  0.         ... 0.         0.         0.        ]
 [0.         0.         0.02585518 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03374591 0.         0.        ]
 [0.         0.         0.         ... 0.         0.01622208 0.        ]
 [0.         0.         0.         ... 0.         0.         0.01392336]]
[[0.02856736 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0312035  0.         ... 0.         0.         0.        ]
 [0.         0.         0.02585518 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03374591 0.         0.        ]
 [0.         0.         0.         ... 0.         0.01622208 0.        ]
 [0.         0.         0.         ... 0.         0.         0.01392336]]
[[0.02856736 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0312035  0.         ... 

LinAlgError: Singular matrix

In [15]:
fac_p_x_w[0,0], fou_p_x_w[0,0], kar_p_x_w[0,0]

NameError: name 'fac_p_x_w' is not defined